Titanic Survival Prediction
==========================
**Titanic_v1** 
타이타닉호 생존에 관련된 변수들을 분석한다
    Test, Train셋 데이터 정의
> pclass : A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

        -> 대체로 나이가 많은 사람들이 1st class를 타는 경향이 보인다.

> age : Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
        
        -> Age column을 확인하고 이상한 값이 있나 확인해봐야겠다.
        
> sibsp : The dataset defines family relations in this way...

        -> 가족들이 같은 배에 타고있는 경우인데 어떻게 해석해야할지? 모르겠다.. 특성엔지니어링이 필요할듯
        
> Sibling : brother, sister, stepbrother, stepsister

        ->  형제들이 있고 없고가 데이터에 영향을 끼치나?
        
> Spouse : husband, wife (mistresses and fiancés were ignored)
        
        -> 배우자가 있는게 영향을 끼치나요? 정부와 약혼자는 제외됌

> parch : The dataset defines family relations in this way...

        -> 부모의 수, 자녀들이 배에 타고 있는경우

> Parent : mother, father

        -> 부모가 있는경우
        
> Child : daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

        -> 몇몇 아이들은 유모랑 같이 탔음 그런애들은 부모들 수에서 제외



In [1]:
import pandas as pd
import numpy as np

In [32]:
train = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/train.csv')
test = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/test.csv')
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [8]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
train.head() # 여기서 Column들을 보고 Categorical한 변수와 Numeric한 변수를 어떻게 처리할지 고민하자

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [11]:
train.info()
# info를 통해 누락된 정보가 있는 column들을 찾아보자
# 여기서는 Age,Cabin이 있고 Embarked는 2개가 빠져있다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


**성별에 따른 생존률을 기준으로 데이터처리**
================

In [12]:
train.groupby('Sex')[['Survived']].mean()
train.pivot_table(values='Survived',index='Sex',aggfunc=np.mean)

,Survived
Sex,
female,0.742038
male,0.188908


In [23]:
train.pivot_table(index='Sex')

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
Sex,,,,,,,
female,27.915709,44.479818,0.649682,431.028662,2.159236,0.694268,0.742038
male,30.726645,25.523893,0.235702,454.147314,2.389948,0.429809,0.188908


In [22]:
train.groupby('Sex')[['Survived']].describe()
# describe로본 min~max는 별로 의미가 없다.

Survived                                             
          count      mean       std  min  25%  50%  75%  max
Sex                                                         
female    314.0  0.742038  0.438211  0.0  0.0  1.0  1.0  1.0
male      577.0  0.188908  0.391775  0.0  0.0  0.0  0.0  1.0

**나이에 따른 생존률**
===========

In [35]:
train.groupby('Age')[['Survived']].mean()
train.pivot_table(values='Survived',index='Age',aggfunc=np.mean)

,Survived
Age,
0.42,1.000000
0.67,1.000000
0.75,1.000000
0.83,1.000000
0.92,1.000000
1.00,0.714286
2.00,0.300000
3.00,0.833333
4.00,0.700000


In [36]:
train.groupby('Age')[['Survived']].describe()
# 나이에 따른 생존관계는 직관적으로 확인하기 어려운듯

Survived                                               
         count      mean       std  min   25%  50%   75%  max
Age                                                          
0.42       1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
0.67       1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
0.75       2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
0.83       2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
0.92       1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
1.00       7.0  0.714286  0.487950  0.0  0.50  1.0  1.00  1.0
2.00      10.0  0.300000  0.483046  0.0  0.00  0.0  0.75  1.0
3.00       6.0  0.833333  0.408248  0.0  1.00  1.0  1.00  1.0
4.00      10.0  0.700000  0.483046  0.0  0.25  1.0  1.00  1.0
5.00       4.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
6.00       3.0  0.666667  0.577350  0.0  0.50  1.0  1.00  1.0
7.00       3.0  0.333333  0.577350  0.0  0.00  0.0  0.50  1.0
8.00       4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0
9.00       8.0  0.250000  0.462910  0.0  0.00  0.0  0.25  1.0
10.00      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
11.00      4.0  0.250000  0.500000  0.0  0.00  0.0  0.25  1.0
12.00      1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
13.00      2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
14.00      6.0  0.500000  0.547723  0.0  0.00  0.5  1.00  1.0
14.50      1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
15.00      5.0  0.800000  0.447214  0.0  1.00  1.0  1.00  1.0
16.00     17.0  0.352941  0.492592  0.0  0.00  0.0  1.00  1.0
17.00     13.0  0.461538  0.518875  0.0  0.00  0.0  1.00  1.0
18.00     26.0  0.346154  0.485165  0.0  0.00  0.0  1.00  1.0
19.00     25.0  0.360000  0.489898  0.0  0.00  0.0  1.00  1.0
20.00     15.0  0.200000  0.414039  0.0  0.00  0.0  0.00  1.0
20.50      1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
21.00     24.0  0.208333  0.414851  0.0  0.00  0.0  0.00  1.0
22.00     27.0  0.407407  0.500712  0.0  0.00  0.0  1.00  1.0
23.00     15.0  0.333333  0.487950  0.0  0.00  0.0  1.00  1.0
...        ...       ...       ...  ...   ...  ...   ...  ...
44.00      9.0  0.333333  0.500000  0.0  0.00  0.0  1.00  1.0
45.00     12.0  0.416667  0.514929  0.0  0.00  0.0  1.00  1.0
45.50      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
46.00      3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
47.00      9.0  0.111111  0.333333  0.0  0.00  0.0  0.00  1.0
48.00      9.0  0.666667  0.500000  0.0  0.00  1.0  1.00  1.0
49.00      6.0  0.666667  0.516398  0.0  0.25  1.0  1.00  1.0
50.00     10.0  0.500000  0.527046  0.0  0.00  0.5  1.00  1.0
51.00      7.0  0.285714  0.487950  0.0  0.00  0.0  0.50  1.0
52.00      6.0  0.500000  0.547723  0.0  0.00  0.5  1.00  1.0
53.00      1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0
54.00      8.0  0.375000  0.517549  0.0  0.00  0.0  1.00  1.0
55.00      2.0  0.500000  0.707107  0.0  0.25  0.5  0.75  1.0
55.50      1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
56.00      4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0
57.00      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
58.00      5.0  0.600000  0.547723  0.0  0.00  1.0  1.00  1.0
59.00      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
60.00      4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0
61.00      3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
62.00      4.0  0.500000  0.577350  0.0  0.00  0.5  1.00  1.0
63.00      2.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0
64.00      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
65.00      3.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
66.00      1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
70.00      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
70.50      1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
71.00      2.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0
74.00      1.0  0.000000       NaN  0.0  0.00  0.0  0.00  0.0
80.00      1.0  1.000000       NaN  1.0  1.00  1.0  1.00  1.0

[88 rows x 8 columns]

In [38]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Feature Engineering**
========
티켓클래스와 성별을 합쳐서 
Survivied랑 어떤 관계인지 확인해 보자


In [44]:
train.pivot_table(('Survived'),['Sex','Pclass'],aggfunc=np.mean)
# WoW first class에 탄 여자는 96%가 살아남았다 우연인가????

Survived
Sex    Pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

대체로 여성이 남성보다 많이 살아남았다는 것을 확인!! 튜닝하기전에 표본으로 삼기위해 Survived컬럼의 True를 여성으로, False를 남성으로 변환하여 제출해 봅니다

In [51]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [52]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
test['Survived'] = (test.Sex == 'female') & (test.Age>0)

In [54]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,False
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,True
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,False
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,False
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,True


In [57]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Survived         0
dtype: int64

In [58]:
test['Survived'].value_counts()

False    291
True     127
Name: Survived, dtype: int64

In [59]:
submission = test[['PassengerId','Survived']].copy()
submission.head()

,PassengerId,Survived
0,892,False
1,893,True
2,894,False
3,895,False
4,896,True


In [60]:
submission['Survived'] = submission['Survived'].astype(int)

In [61]:
submission.head(100)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [62]:
submission.to_csv('../titanic/submission.csv',index = False)

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 5EEC-C469

 C:\Users\sangkyu\git\kaggle\titanic 디렉터리

2019-06-07  오후 01:04    <DIR>          .
2019-06-07  오후 01:04    <DIR>          ..
2019-06-07  오전 11:53    <DIR>          .ipynb_checkpoints
2019-06-06  오후 11:55           335,319 ensamble_test.ipynb
2019-06-07  오전 11:30    <DIR>          input
2019-06-07  오후 01:03             3,258 submission.csv
2019-06-07  오전 11:53           109,775 titanic_v1.ipynb
2019-06-03  오전 11:49            24,771 titanic_v2.ipynb
2019-06-03  오전 11:49            25,957 titanic_v3.ipynb
2019-06-03  오전 11:49           123,485 titanic_v4.ipynb
2019-06-07  오후 01:04           101,576 Untitled.ipynb
               7개 파일             724,141 바이트
               4개 디렉터리  37,062,729,728 바이트 남음
